<h1>MNIST DIGIT CLASSIFICATION USING A PYTORCH NEURAL NETWORK</h1>

In [23]:
#Import necessary modules
import torch
from torch import nn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

device="cuda" if torch.cuda.is_available() else "cpu" #Set the device


In [5]:
#Load The Datasets
trainSet=pd.read_csv(r"D:\Datasets\MNIST\mnist_train.csv")
testSet=pd.read_csv(r"D:\Datasets\MNIST\mnist_test.csv")

#Since we have separate datasets for training and testing , we donot need to perform train test split

In [6]:
testSet.head()

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [56]:
#Converting Df Into tensors
train_x,train_y=torch.from_numpy(np.array(trainSet.iloc[:,1:]))/255.0,torch.from_numpy(np.array(trainSet.iloc[:,0]))
train_x.to(device)
train_y.to(device)
test_x,test_y=torch.from_numpy(np.array(testSet.iloc[:,1:]))/255.0,torch.from_numpy(np.array(testSet.iloc[:,0]))
print(train_x.shape,train_y.shape)


#One hot encoding the train_y
def oneHEncoding(data,size):
    return torch.eye(size)[data];
train_y=oneHEncoding(train_y,10)






torch.Size([60000, 784]) torch.Size([60000])


In [99]:
#Creating the model
class DigitClassifier(nn.Module):
    def __init__(self,lr,epochs,no_of_neurons):
        super().__init__()
        self.lr=lr
        self.epochs=epochs
        self.l1=nn.Linear(784,no_of_neurons)
        self.l2=nn.Linear(no_of_neurons,no_of_neurons)
        self.l3=nn.Linear(no_of_neurons,10)
        self.relu=nn.ReLU()
        self.optimizer=torch.optim.SGD(params=self.parameters(),lr=self.lr,momentum=0.9)
        torch.nn.init.kaiming_uniform_(self.l1.weight, nonlinearity='relu')
        torch.nn.init.kaiming_uniform_(self.l2.weight, nonlinearity='relu')
        torch.nn.init.kaiming_uniform_(self.l3.weight, nonlinearity='relu')
        
    def forward(self,x):
        x=self.relu(self.l1(x))
        x=self.relu(self.l2(x))
        return self.l3(x)

    def fit(self,x,y):
        self.train()
        loss_fn=nn.CrossEntropyLoss()
        for i in range(self.epochs):
            self.train()
            print(f"Iteration {i}")
            
            y_pred=self.forward(x)
            loss=loss_fn(y_pred,y)
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            self.eval()
            with torch.inference_mode():
                if(i%10==0):
                    print(f"Loss={loss}")
            self.train()

                    
                
            

        

In [100]:
#Creating an instance of  the model
torch.manual_seed(42)
model=DigitClassifier(0.1,1000,16)

model.fit(train_x,train_y)




Iteration 0
Loss=2.416447877883911
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
Iteration 10
Loss=1.579172968864441
Iteration 11
Iteration 12
Iteration 13
Iteration 14
Iteration 15
Iteration 16
Iteration 17
Iteration 18
Iteration 19
Iteration 20
Loss=0.8556860685348511
Iteration 21
Iteration 22
Iteration 23
Iteration 24
Iteration 25
Iteration 26
Iteration 27
Iteration 28
Iteration 29
Iteration 30
Loss=0.638761043548584
Iteration 31
Iteration 32
Iteration 33
Iteration 34
Iteration 35
Iteration 36
Iteration 37
Iteration 38
Iteration 39
Iteration 40
Loss=0.4863605201244354
Iteration 41
Iteration 42
Iteration 43
Iteration 44
Iteration 45
Iteration 46
Iteration 47
Iteration 48
Iteration 49
Iteration 50
Loss=0.4108235538005829
Iteration 51
Iteration 52
Iteration 53
Iteration 54
Iteration 55
Iteration 56
Iteration 57
Iteration 58
Iteration 59
Iteration 60
Loss=0.36947065591812134
Iteration 61
Iteration 62
Iteration 63
Iteration 64

In [101]:
#Testing the model  on unseen data


model.eval()
y_pred=model.forward(test_x)
y_labels=torch.argmax(y_pred,dim=1)
correct=torch.sum(y_labels==test_y)
print(f"Accuracy:{correct/100.0}")

Accuracy:95.08999633789062


### As Compared To The Raw Model before , this model performs slightly better.